#  AAS JSON →  Knowledge GraphConversion Pipeline

This notebook demonstrates the transformation of:
- an **ISO 63278 Asset Administration Shell (AAS) JSON** 
- via a **SPARQL CONSTRUCT mapping**  
- into  **OWL-based Knowledge Graph (KG)**  

Pipeline Steps:

1. Load AAS instance
2. Validate JSON against AAS Meta-Model schema
3. Convert AAS JSON -> RDF (using `py-aas-rdf`)
4. Apply SPARQL mapping → AAS RDF
5. Save output RDF



## Imports & Environment Setup

In [1]:
import json
from rdflib import Graph, Namespace
from py_aas_rdf.models.submodel import Submodel
from pyoxigraph import RdfFormat, Store
import rdflib
from pathlib import Path
from dsms import DSMS


## Variable initialization and load configuration from .env to initialize DSMS instance

In [3]:
# -----------------------------
# Paths & constants
# -----------------------------
AAS_JSON_PATH = Path("./input/InspectionDocumentsOfSteelProducts.json")
INSPECTION_DOCUMENT_MAPPING_PATH = Path("./input/InspectionDocumentsOfSteelProducts_AAS2KG_mapping.sparql")
OUTPUT_DIR = Path("./output")

PLACEHOLDER = "PLACEHOLDER"
UNIQUE_ID = "test123"

AAS_NS = Namespace("https://admin-shell.io/aas/3/0/")
BASE_URI = "https://example.org/aas/"

# DSMS setup
dsms = DSMS(env=".env")

/home/kiran/work/mapro/KG2AAS-prototype/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmdx.materials-data.space'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kiran/work/mapro/KG2AAS-prototype/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmdx.materials-data.space'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kiran/work/mapro/KG2AAS-prototype/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmdx.materials-data.space'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

## Apply AAS2KG pipeline

In [4]:

# Load AAS JSON
with open(AAS_JSON_PATH, "r", encoding="utf-8") as f:
    aas_json = json.load(f)
        
# Run for each submodel in inspection document
for sm_json in aas_json.get("submodels", []):
    rdf_graph = Graph()
    rdf_graph.bind("aas", "https://admin-shell.io/aas/3/0/")
    
    # Validate submodel using py-aas-rdf library
    submodel = Submodel.model_validate(sm_json)
    # Convert submodel to RDF
    submodel.to_rdf(rdf_graph, base_uri="https://example.org/aas/",  id_strategy="base64-url-encode")
    ttl_output = rdf_graph.serialize(format="turtle")
    
    # Apply SPARQL construct for specific submodels 
    if sm_json.get("idShort") == "InspectionDocumentsOfSteelProducts":
        with open(INSPECTION_DOCUMENT_MAPPING_PATH, "r", encoding="utf-8") as f:
            mapping_query = f.read().replace(PLACEHOLDER, UNIQUE_ID)

            store = Store()

            store.load(ttl_output, format=RdfFormat.TURTLE)
            mapping_result = store.query(mapping_query)

            # Serialize the result from Oxigraph (returns bytes)
            mapped_bytes = mapping_result.serialize(format=RdfFormat.TURTLE)

            # --- Parse into rdflib.Graph for prefix binding ---
            g = rdflib.Graph()
            g.parse(data=mapped_bytes, format="turtle")

            g.bind("aas", "https://admin-shell.io/aas/3/0/")
            g.bind("ex", "http://www.example.org/#")
            g.bind("pmd", "https://w3id.org/pmd/co")
            g.bind("obo", "http://purl.obolibrary.org/obo")

            # Serialize with prefixes applied
            final_ttl = g.serialize(format="turtle")
            
            print("===============InspectionDocumentsOfSteelProducts=======================")
            print(final_ttl)
            # Write to file
            with open("./output/InspectionDocumentsOfSteelProducts_AAS.ttl", "w", encoding="utf-8") as f:
                f.write(final_ttl)

            print("✓ Applied SPARQL mapping and namespace bindings.")

===============InspectionDocumentsOfSteelProducts=======================
@prefix ex: <http://www.example.org/#> .
@prefix ns1: <http://purl.obolibrary.org/obo/> .
@prefix ns2: <https://w3id.org/pmd/co/> .

ex:test123_chem_comp a ns2:PMD_0000551 ;
    ns1:RO_0000080 ex:test123_material ;
    ns2:PMD_0000004 ex:test123_chem_comp_spec .

ex:test123_mass_proportion_carbon a ns2:PMD_0020102 ;
    ns2:PMD_0000077 ex:test123_fraction_carbon ;
    ns2:PMD_0025999 ex:test123_some_carbon .

ex:test123_mass_proportion_silicon a ns2:PMD_0020102 ;
    ns2:PMD_0000077 ex:test123_fraction_silicon ;
    ns2:PMD_0025999 ex:test123_some_silicon .

ex:test123_chem_comp_spec a ns2:PMD_0025002 ;
    ns1:RO_0002351 ex:test123_fraction_carbon,
        ex:test123_fraction_silicon .

ex:test123_some_carbon a ns2:PMD_0020049 ;
    ns1:BFO_0000050 ex:test123_material .

ex:test123_some_silicon a ns2:PMD_0020050 ;
    ns1:BFO_0000050 ex:test123_material .

ex:test123_fraction_carbon a ns2:PMD_0025997 ;
    ns1:IA